In [19]:
# import packages, set options
import pandas as pd
import numpy as np
import math
pd.set_option("display.max.columns", None)
pd.options.mode.chained_assignment = None

In [2]:
# Set the years to apply the programme to - format is 20yy/yy+1, e.g 17 will give 2017/18 season
betting_year=17
data_year=betting_year-1

In [3]:
# Define function for poisson distribution:
def poisson(actual, mean):
    return(mean**actual*math.exp(-mean))/math.factorial(actual)

In [4]:
# Load the historical dataset that expected values will be created from
data=pd.read_csv('data/EPL/%d.csv'%(data_year)) 

In [5]:
# Create a list of teams for the mean goals to be stored for
teamlist=[]
for i in range(len(data)):
    if data.HomeTeam[i] not in teamlist:
        teamlist.append(data.HomeTeam[i])
teamlist.sort()

In [6]:
# Create dataframe to store mean home and away goals in
columns=['home_goals', 'away_goals', 'home_conceded', 'away_conceded',
           'home_games', 'away_games','total_games', 'alpha_h','alpha_a']
teaminfo= pd.DataFrame(index=teamlist, columns=columns)
teaminfo[(columns)]=0

In [7]:
# Make calculations for the mean values
for i in range(len(data)):
    teaminfo['home_games'][(data.HomeTeam[i])]+=1
    teaminfo['away_games'][(data.AwayTeam[i])]+=1
    teaminfo['home_goals'][(data.HomeTeam[i])]+=data.FTHG[i]
    teaminfo['away_goals'][(data.AwayTeam[i])]+=data.FTAG[i]
    
teaminfo['alpha_h']=teaminfo['home_goals']/teaminfo['home_games']
teaminfo['alpha_a']=teaminfo['away_goals']/teaminfo['away_games']
teaminfo['total_games']=teaminfo['home_games']+teaminfo['away_games']

In [15]:
# Load the dataset of fixtures for which you want to bet on, and add columns for some metrics
season=pd.read_csv('data/EPL/%d.csv'%(betting_year))
add_columns=['p_win','p_draw','p_draw','p_loss','sum_probs',
     'ev_win','ev_draw','ev_loss','promoted']
for i in add_columns:
    season[i]=np.zeros((len(season)))

In [20]:
# Removes the matches for teams that have just been promoted, as don't have mean goals for them
for i in range(len(season)):
    if season.HomeTeam[i] not in teamlist or season.AwayTeam[i] not in teamlist:
        season.promoted[i]=1
season.drop(season[season.promoted==1.0].index,inplace=True)
season.reset_index(drop=True, inplace=True)

In [21]:
# Set maxscore to the max goals for each team you want to calculate probability of. e.g =11 will calculate up to
    # 10-10
# Calculates the probability of each result, and then adds the relevant ones together to get p of each outcome
maxscore = 11
for game in range(len(season)):    
    probs=pd.DataFrame(index=range(maxscore**2),columns=['homescore','awayscore','probability'])
    index_counter=0
    for i in range(maxscore):
        for j in range(maxscore):
            prob = poisson(i, teaminfo['alpha_h'][season.HomeTeam[game]]) * poisson(j,teaminfo['alpha_a'][season.AwayTeam[game]])
            probs.homescore[index_counter]=i
            probs.awayscore[index_counter]=j               
            probs.probability[index_counter]=prob
            index_counter+=1
    p_win=0
    p_loss=0
    p_draw=0
    for i in range(len(probs)):
        if probs.homescore[i]>probs.awayscore[i]:
            p_win+=probs.probability[i]
        if probs.homescore[i]<probs.awayscore[i]:
            p_loss+=probs.probability[i]
        if probs.homescore[i]==probs.awayscore[i]:
            p_draw+=probs.probability[i] 
    season['p_win'][game]=p_win
    season['p_draw'][game]=p_draw
    season['p_loss'][game]=p_loss
    season['sum_probs'][game]=np.sum((p_win,p_draw,p_loss))

In [22]:
# Calculates and adds the expected value of each bet to the data set. Use the bet 365 odds here
for game in range(len(season)):
    season.ev_win[game]=(season.p_win[game]*(season.B365H[game]-1))-(1-season.p_win[game])
    season.ev_draw[game]=(season.p_draw[game]*(season.B365D[game]-1))-(1-season.p_draw[game])
    season.ev_loss[game]=(season.p_loss[game]*(season.B365A[game]-1))-(1-season.p_loss[game])

In [23]:
# Define some variables
starting_bankroll=100
bankroll=starting_bankroll
wager=5    # bet per game
incorrect=0 # counters for track record
correct=0
threshold=0 # the minimum expected value of a bet in order to place one
betcounter=incorrect+correct
games=len(season)

In [24]:
# Carrying out the betting
# If you want to check that these next lines are working how they should, unsilence the print commands. 
for game in range(games):
    result=0
    ev_max=max(season.ev_win[game],season.ev_draw[game],season.ev_loss[game]) 
    if season.ev_win[game]==ev_max and season.ev_win[game]>threshold:
        team_bet=season.HomeTeam[game]
        if season.FTHG[game]>season.FTAG[game]:
            betvalue=wager*(season.B365H[game]-1)
            result='won'
            correct+=1
        else:
            betvalue=-wager
            result='lost'
            incorrect+=1
        bankroll+=betvalue
        #print("Bet",season.HomeTeam[game],'vs',season.AwayTeam[game],':backed',team_bet)
        #print('home scored',season.FTHG[game], 'away scored',season.FTAG[game])
        #print('Bet',result)
        #print('Bankroll=',bankroll)
    elif season.ev_draw[game]==ev_max and season.ev_draw[game]>threshold:
        team_bet="draw"
        if season.FTHG[game]==season.FTAG[game]:
            betvalue=wager*(season.B365D[game]-1)
            result='won'
            correct+=1
        else:
            betvalue=-wager
            result='lost'
            incorrect+=1
        bankroll+=betvalue
        #print("Bet",season.HomeTeam[game],'vs',season.AwayTeam[game],':backed',team_bet)
        #print('home scored',season.FTHG[game], 'away scored',season.FTAG[game])
        #print('Bet',result)
        #print('Bankroll=',bankroll)
    elif season.ev_loss[game]==ev_max and season.ev_loss[game]>threshold:
        team_bet=season.AwayTeam[game]
        if season.FTHG[game]<season.FTAG[game]:
            betvalue=wager*(season.B365A[game]-1)
            result='won'
            correct+=1
        else:
            betvalue=-wager
            result='lost'
            incorrect+=1
        bankroll+=betvalue
        #print("Bet",season.HomeTeam[game],'vs',season.AwayTeam[game],':backed',team_bet)
        #print('home scored',season.FTHG[game], 'away scored',season.FTAG[game])
        #print('Bet',result)
        #print('Bankroll=',bankroll)

In [25]:
# Returning the performance metrics of the betting        
betcounter=incorrect+correct
ROI = ((bankroll - starting_bankroll) / (wager * (betcounter)))-1
ROI="{:.2%}".format(ROI)
print('Made',betcounter,'bets out of a maximum',len(season))
print(correct,'were correct',incorrect,'were incorrect')
print('Total money betted=',wager*betcounter)
print('ROI=',ROI)

Made 270 bets out of a maximum 272
66 were correct 204 were incorrect
Total money betted= 1350
ROI= -107.94%
